In [1]:
from __future__ import annotations

from pathlib import Path

import pandas as pd

from wufam.config.trading_config import TradingConfig
from wufam.dataset import Dataset
from wufam.strategies.optimization_data import TrainingData, PredictionData

PATH = Path("../data/kf_data")
START = "1970-01-01"
END = "2024-12-31"
DATASET = Dataset.BM_6

In [2]:
factors_df = pd.read_csv(
    PATH / DATASET,
    skiprows=26_045,
    skipfooter=104_126-52_070,
    index_col=0,
    engine="python"
)
factors_df.index = pd.to_datetime(factors_df.index, format="%Y%m%d")
factors_df = factors_df.loc[START:END]
factors_df = factors_df / 100
factors_df.head()

,SMALL LoBM,ME1 BM2,SMALL HiBM,BIG LoBM,ME2 BM2,BIG HiBM
1970-01-02,0.0459,0.0395,0.0334,0.0107,0.0152,0.0246
1970-01-05,0.0260,0.0222,0.0240,0.0026,0.0077,0.0123
1970-01-06,-0.0076,-0.0038,-0.0009,-0.0090,-0.0077,-0.0065
1970-01-07,0.0054,0.0055,0.0037,-0.0008,-0.0027,-0.0025
1970-01-08,0.0051,0.0062,0.0027,0.0001,-0.0002,-0.0017


In [3]:
from wufam.strategies.heuristics.equally_weighted import EWStrategy
from wufam.strategies.estimated.mean_var import MeanVariance
from wufam.strategies.estimated.min_var import MinVariance

trading_config = TradingConfig(total_exposure=1)

ew_strategy = EWStrategy()

In [4]:
from wufam.estimation.mean.sample_mu_estimator import SampleMuEstimator
from wufam.estimation.covariance.sample_cov_estimator import SampleCovEstimator


mv_strategy = MeanVariance(
    mu_estimator=SampleMuEstimator(),
    cov_estimator=SampleCovEstimator(),
    trading_config=trading_config,
    window_size=None
)

In [5]:
from wufam.estimation.covariance.shrinkage.pca_cov_estimator import PCACovEstimator

min_var_strategy = MinVariance(
    cov_estimator=PCACovEstimator(),
    trading_config=trading_config,
    window_size=None,
)

In [6]:
training_data = TrainingData(
    simple_excess_returns=factors_df,
)

ew_strategy.fit(training_data)
mv_strategy.fit(training_data)
min_var_strategy.fit(training_data)

In [7]:
prediction_data = PredictionData()

ew_weights = ew_strategy(prediction_data)
mv_weights = mv_strategy(prediction_data)
min_var_weights = min_var_strategy(prediction_data)

In [8]:
ew_weights.head()

,SMALL LoBM,ME1 BM2,SMALL HiBM,BIG LoBM,ME2 BM2,BIG HiBM
0,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667


In [9]:
mv_weights.head()

,SMALL LoBM,ME1 BM2,SMALL HiBM,BIG LoBM,ME2 BM2,BIG HiBM
0,-7.671588,-2.310579,16.489084,5.833693,-10.517793,-0.822816


In [10]:
min_var_weights.head()

,SMALL LoBM,ME1 BM2,SMALL HiBM,BIG LoBM,ME2 BM2,BIG HiBM
0,-1.435176,-0.488751,2.653299,0.945826,0.995038,-1.670235
